In [1]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd

import json

from bouter import EmbeddedExperiment
from fimpylab.core.lightsheet_experiment import LightsheetExperiment
from bouter.utilities import predictive_tail_fill
TURN_BIAS = 0.239

from bouterin.plots.stimulus_log_plot import get_paint_function

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt

In [3]:
master_path =  Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v28\2d")
master_path =  Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\ls_fixed\spont_plus_v13\huc")
master_path =  Path(r"\\funes\shared\experiments\E0040_motions_cardinal\v29_29p1s\ls\2sec")
master_path = Path(r"Z:\Hagar\older fish\h2b")

fish_list = list(master_path.glob("*_f*"))
fish_list

[WindowsPath('Z:/Hagar/older fish/h2b/230501_f3_v13'),
 WindowsPath('Z:/Hagar/older fish/h2b/230501_f3_v19'),
 WindowsPath('Z:/Hagar/older fish/h2b/230501_f4_v13'),
 WindowsPath('Z:/Hagar/older fish/h2b/230501_f4_v19'),
 WindowsPath('Z:/Hagar/older fish/h2b/230501_f5_v13'),
 WindowsPath('Z:/Hagar/older fish/h2b/230501_f5_v19'),
 WindowsPath('Z:/Hagar/older fish/h2b/230501_f6_v13'),
 WindowsPath('Z:/Hagar/older fish/h2b/230501_f6_v19'),
 WindowsPath('Z:/Hagar/older fish/h2b/230502_f0_v13'),
 WindowsPath('Z:/Hagar/older fish/h2b/230502_f0_v19'),
 WindowsPath('Z:/Hagar/older fish/h2b/230502_f1_v13'),
 WindowsPath('Z:/Hagar/older fish/h2b/230502_f1_v19'),
 WindowsPath('Z:/Hagar/older fish/h2b/230502_f2_v13'),
 WindowsPath('Z:/Hagar/older fish/h2b/230502_f2_v19'),
 WindowsPath('Z:/Hagar/older fish/h2b/230502_f3_v13'),
 WindowsPath('Z:/Hagar/older fish/h2b/230502_f3_v19'),
 WindowsPath('Z:/Hagar/older fish/h2b/230502_f4_v13'),
 WindowsPath('Z:/Hagar/older fish/h2b/230502_f4_v19'),
 WindowsPa

In [4]:
for fish in fish_list:
    print(fish)
    try:
        #fn = 2
        if not (fish / "bouts_df.h5").exists():
            #exp = EmbeddedExperiment(fish)
            exp = LightsheetExperiment(fish)
            fn = exp.fn
            beh_df = exp.behavior_log
            print("T")
            #theta_mat = beh_df.loc[:, [f"theta_0{i}" for i in range(9)]].values
            theta_mat = beh_df.iloc[:, 1:-1].values

            n_segments = np.shape(theta_mat)[1]
            beh_df.loc[:, [f"theta_0{i}" for i in range(n_segments)]] = predictive_tail_fill(
                theta_mat
            )
            print(n_segments)

            try:
                beh_df["tail_sum"] = (beh_df["theta_07"] + beh_df["theta_08"]) - (
                    beh_df["theta_00"] + beh_df["theta_01"]
                )
            except:
                try:
                    beh_df["tail_sum"] = (beh_df["theta_07"] + beh_df["theta_06"]) - (
                        beh_df["theta_00"] + beh_df["theta_01"]
                    )
                except:
                    beh_df["tail_sum"] = (beh_df["theta_05"] + beh_df["theta_04"]) - (
                    beh_df["theta_00"] + beh_df["theta_01"]
                )

            bouts_df = exp.get_bout_properties(vigor_threshold=0.05)
            # Compute bout index in behavior trace:
            bouts_df["idx"] = [
                np.argmin((beh_df["t"] - bouts_df.loc[i, "t_start"]).abs())
                for i in bouts_df.index
            ]
            bouts_df["fid"] = fish.name

            bouts_df["idx_imaging"] = np.round(bouts_df["t_start"] * fn).astype(np.int)

            bouts_df["direction"] = "fw"
            bouts_df.loc[(bouts_df["bias"] > TURN_BIAS), "direction"] = "rt"
            bouts_df.loc[(bouts_df["bias"] < -TURN_BIAS), "direction"] = "lf"

            fl.save(fish / "bouts_df.h5", bouts_df)
    except:
        print("Error. Stupid fish wasting my time.")

Z:\Hagar\older fish\h2b\230501_f3_v13
Z:\Hagar\older fish\h2b\230501_f3_v19
Z:\Hagar\older fish\h2b\230501_f4_v13
Z:\Hagar\older fish\h2b\230501_f4_v19
Z:\Hagar\older fish\h2b\230501_f5_v13
Z:\Hagar\older fish\h2b\230501_f5_v19
Z:\Hagar\older fish\h2b\230501_f6_v13
Z:\Hagar\older fish\h2b\230501_f6_v19
Z:\Hagar\older fish\h2b\230502_f0_v13
T
7


C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Z:\Hagar\older fish\h2b\230502_f0_v19
T
7


C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Z:\Hagar\older fish\h2b\230502_f1_v13
T
7


...

C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Z:\Hagar\older fish\h2b\230502_f5_v13
T
7


C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


Z:\Hagar\older fish\h2b\230502_f5_v19
T
7


C:\Users\lavian\anaconda3\envs\rplab\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [ ]:
theta_mat = beh_df.iloc[:, 1:-1]

In [ ]:
beh_df

In [ ]:
master_path = Path(r"Z:\Hagar\E0040\v31\pre ablation")
fish_list = list(master_path.glob("*_f*"))

In [5]:
for fish in fish_list:
    print(fish)
    #try:
    if not (fish / "tail_trace.jpg").exists():
        #exp = EmbeddedExperiment(fish)
        exp = LightsheetExperiment(fish)
        beh_df = exp.behavior_log
        fig = plt.figure(figsize=(12,3))
        plt.plot(beh_df.t, beh_df.tail_sum, color='k')
        plt.xlabel('Time (sec)')
        fig.savefig(fish / "tail_trace.jpg")
            
    #except:
    #    print("Error. Stupid fish wasting my time.")

Z:\Hagar\older fish\h2b\230501_f3_v13
Z:\Hagar\older fish\h2b\230501_f3_v19
Z:\Hagar\older fish\h2b\230501_f4_v13
Z:\Hagar\older fish\h2b\230501_f4_v19
Z:\Hagar\older fish\h2b\230501_f5_v13
Z:\Hagar\older fish\h2b\230501_f5_v19
Z:\Hagar\older fish\h2b\230501_f6_v13
Z:\Hagar\older fish\h2b\230501_f6_v19
Z:\Hagar\older fish\h2b\230502_f0_v13


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Z:\Hagar\older fish\h2b\230502_f0_v19


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Z:\Hagar\older fish\h2b\230502_f1_v13


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Z:\Hagar\older fish\h2b\230502_f5_v13


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Z:\Hagar\older fish\h2b\230502_f5_v19


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
master_path = Path(r"Z:\Hagar\E0040\2p ablations\E0040V13")
fish_list = list(master_path.glob("*_f*"))

In [ ]:
for fish in fish_list:
    print(fish)
    #try:
    if not (fish / "tail trace with stim.jpg").exists():
        #exp = EmbeddedExperiment(fish)
        exp = LightsheetExperiment(fish)
        beh_df = exp.behavior_log
        fig = plt.figure(figsize=(12,3))
        
        stimulus_log = exp.stimulus_log
        stim_value, t_values = get_paint_function(stimulus_log, 'E0040_motions_cardinal')
        stim_value = stim_value / 255
        num_stim = np.shape(stim_value)[0]

        #t_values *= exp.fn
        for i in range(num_stim):
            plt.axvspan(
                t_values[i, 0],
                t_values[i, 1],
                facecolor=[
                    stim_value[i, 0],
                    stim_value[i, 1],
                    stim_value[i, 2],
                ],
                alpha=0.7,
            )
    
        plt.plot(beh_df.t, beh_df.tail_sum, color='k')
        plt.xlabel('Time (sec)')
        fig.savefig(fish / "tail trace with stim.jpg")
            
    #except:
    #    print("Error. Stupid fish wasting my time.")